In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

apartmentsRaw = spark.read.format("json").option("multiLine", "true").load("dbfs:/FileStore/raw/Apartments/").withColumn("apartment", F.explode(F.col("apartments"))).select("apartment.*")

print("Raw: " + str(apartmentsRaw.count()))

deduplicated = apartmentsRaw.withColumn("rn", F.row_number().over(Window().partitionBy(["id"]).orderBy(F.desc("last_time_up")))).filter(F.col("rn") == 1).drop("rn")

print("Deduplicated: " + str(deduplicated.count()))

assert(deduplicated.groupBy("id").count().filter(F.col("count") > 1).count() == 0)

deduplicated.withColumn("year", F.year(F.current_date())).withColumn("month", F.month(F.current_date())).withColumn("day", F.dayofmonth(F.current_date())).write.mode("append").partitionBy(["year", "month", "day"]).parquet("dbfs:/FileStore/cleaned/Apartments")